# Create a dataframe of NTRL Lab Analysis Results month by month

In [ ]:
# This program helps you generate the dataframe for each month per area of sampling

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

#used in the analysis results data. cleans up the sampling time entry
#---------------------
#
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
#
#-------------------
#
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname

#----------------------
#
#generate_df_from_analysis: this function creates dataframe by getting laboratory results by taking the excel values from
#    every row in a specified analysis column. An example of analysis column is the column for Zn AAS analysis for NTRL.
#    The critical part of this code is in making sure that the dates and time from the laboratory analysis excel file are
#    parsed and converted into the correct datetime format.
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df
#-----------------------------------------------------End----------------------------------------------------------------
#
#
#------------------------------------------------------------------------------------------------------------------------
#
def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#
#----------------------------------------------------End-----------------------------------------------------------------
#
#
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df
#----------------------------------------------------End---------------------------------------------------------------
#
#
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df
#
#----------------------------------------------------End-----------------------------------------------------------------
#
#
def append_monthsDF(dfsListmonthly): #takes the list of dataframes from monthly lab results of one sampling area
    df = dfsListmonthly[0]
    for i in range(1,len(dfsListmonthly)):
        df = df.append(dfsListmonthly[i])
    try:
        df.set_index('DATETIME', inplace=True)
    except:
        pass
    return df  
#
#
#
#------------------------------------------------------End-------------------------------------------------------------
#
#
def generate_combined_df_by_month(month_WB, sample_area, analysis_type):
    df_List = []
    for i in analysis_type:
        df_analysis = month_WB.generate_df_from_analysis(sample_area, i)
        df_List.append(df_analysis)
    for i in df_List:
        df = join_DF(df_List)
    return df

#
#
#----------------------------------------------------End-----------------------------------------------------------------
#
#  make_df_for_each_FT01_by_month_then_export_excel: This function exports the 
def make_df_for_each_FT01_by_month_then_export_excel(monthNumber, analysis_type_FT01AtoF_list):
    month_ = LabResults(currentmonthyr_list[monthNumber-1], filepath_list[monthNumber-1])
    letters = 'ABCDEF'
    FT01_months_df = []
    for i in range(0,6):
        FT01_ = generate_combined_df_by_month(month_, '105FT01', analysis_type_FT01AtoF_list[i])
        FT01_.to_excel(r'C:\Users\v.t.flores\Documents\FT01{}_2020_0{}.xlsx'.format(letters[i], monthNumber))
#
#
#---------------------------------------------------End-------------------------------------------------------------------
#
# load_changeColtitle_join_105FT01: This function works closely with the function 'make_df_for_each_FT01_by_month_then_export_excel'
def load_changeColtitle_join_105FT01(monthNumber): #this code will return a dataframe
    FT01_DF_list_by_filter = []
    letters = 'ABCDEF'
    for i in range(0, 6):
        FT01_ = pd.read_excel(r'C:\Users\v.t.flores\Documents\FT01{}_2020_0{}.xlsx'.format(letters[i], monthNumber))
        FT01_.set_index('DATETIME', inplace=True)
        FT01_.columns = [letters[i]+'_Zn',
                         letters[i]+'_ntu',
                         letters[i]+'_Zn_dr']
        FT01_DF_list_by_filter.append(FT01_)
    return join_DF(FT01_DF_list_by_filter)    
#
#
#--------------------------------------------------End-------------------------------------------------------------------

In [ ]:
#monthyear
currentmonthyr_01 = datetime.date(2020,1,1)
currentmonthyr_02 = datetime.date(2020,2,1)
currentmonthyr_03 = datetime.date(2020,3,1)
currentmonthyr_04 = datetime.date(2020,4,1)
currentmonthyr_05 = datetime.date(2020,5,1)
currentmonthyr_06 = datetime.date(2020,6,1)
currentmonthyr_07 = datetime.date(2020,7,1)

currentmonthyr_list = [currentmonthyr_01,
                       currentmonthyr_02,
                       currentmonthyr_03, 
                       currentmonthyr_04,
                       currentmonthyr_05,
                       currentmonthyr_06,
                       currentmonthyr_07]
#filepath
filepath_01 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JANUARY 2020\1) JANUARY_2020.xlsx'
filepath_02 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\FEBRUARY 2020\2) FEBRUARY _2020.xlsx'
filepath_03 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\MARCH 2020\3) March _2020.xlsx'
filepath_04 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\APRIL 2020\4) April _2020.xlsx'
filepath_05 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\MAY 2020\5) MAY_2020.xlsx'
filepath_06 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JUNE 2020\6) June _2020.xlsx'
filepath_07 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\7) July _2020.xlsx'

filepath_list = [filepath_01,
                 filepath_02,
                 filepath_03,
                 filepath_04,
                 filepath_05,
                 filepath_06,
                 filepath_07]

#list_lab_index for 105TK05
deznTK05_Pb = ['B', 'C', 'E', '105TK05_Pb']
deznTK05_Zn =  ['B', 'C', 'F', '105TK05_Zn']
deznTK05_Cu = ['B', 'C', 'G', '105TK05_Cu']
deznTK05_Ni = ['B', 'C', 'H', '105TK05_Ni']
deznTK05_Co = ['B', 'C', 'I', '105TK05_Co']
deznTK05_Fe2 = ['B', 'C', 'J', '105TK05_Fe']
deznTK05_Mn = ['B', 'C', 'K', '105TK05_Mn']
deznTK05_Cr = ['B', 'C', 'L', '105TK05_Cr']
deznTK05_Ca = ['B', 'C', 'M', '105TK05_Ca']
deznTK05_Si = ['B', 'C', 'N', '105TK05_Si']
deznTK05_Al = ['B', 'C', 'O', '105TK05_Al']
deznTK05_Mg = ['B', 'C', 'P', '105TK05_Mg']
deznTK05_pH = ['B', 'C', 'AF', '105TK05_pH']
deznTK05_ORP = ['B', 'C', 'AG', '105TK05_ORP']
deznTK05_PS90 = ['B', 'C', 'AP', '105TK05_PS90']

#list_lab_index for 105TK03
deznTK03_Zn = ['B', 'C', 'D', '105TK03_Zn']
deznTK03_Zn_change =  ['B', 'C', 'BB', '105TK03_Zn_Change_Ratio']
deznTK03_TK05_Zn = ['B', 'C', 'BD', '105TK05_Zn']
deznTK03_TK01_Zn = ['B', 'C', 'BE', '106TK01_Zn']
deznTK03_Zn_Ni = ['B', 'C', 'BF', '106TK01_Zn/Ni']
deznTK03_Zn_Droprate = ['B', 'C', 'BG', '106TK01_Zn_DropRate']
deznTK03_VE04Zn_prcnt = ['B', 'C', 'BH', '106VE04_%Zn']
deznTK03_TH01Zn_prcnt = ['B', 'C', 'BI', '106TH01_%Zn']
deznTK03_NTRL_Ni = ['B', 'C', 'BJ', '104PU01_Ni'] #not true for january, feb, mar 2020 resulst
deznTK03_TK05_Ni = ['B', 'C', 'BK', '105TK05_Ni'] #not true for january, feb, mar 2020 results
deznTK03_Ni_Loss_TK05 = ['B', 'C', 'BL', 'Ni_Loss_TK05'] #not true for january, feb, mar 2020 results

analysis_type_list_tk03 = [deznTK03_Zn,
                           deznTK03_Zn_change,
                           deznTK03_TK05_Zn,
                           deznTK03_TK01_Zn,
                           deznTK03_Zn_Ni,
                           deznTK03_Zn_Droprate,
                           deznTK03_VE04Zn_prcnt,
                           deznTK03_TH01Zn_prcnt,
                           deznTK03_NTRL_Ni,
                           deznTK03_TK05_Ni,
                           deznTK03_Ni_Loss_TK05]

analysis_type_list_tk03_jan = [deznTK03_Zn,
                               deznTK03_Zn_change,
                               deznTK03_TK05_Zn,
                               deznTK03_TK01_Zn,
                               deznTK03_Zn_Ni,
                               deznTK03_Zn_Droprate,
                               deznTK03_VE04Zn_prcnt,
                               deznTK03_TH01Zn_prcnt]

analysis_type_list_tk05 = [deznTK05_Pb,
                           deznTK05_Zn,
                           deznTK05_Cu,
                           deznTK05_Ni,
                           deznTK05_Co,
                           deznTK05_Fe2,
                           deznTK05_Mn,
                           deznTK05_Cr,
                           deznTK05_Ca,
                           deznTK05_Si,
                           deznTK05_Al,
                           deznTK05_Mg,
                           deznTK05_pH,
                           deznTK05_ORP,
                           deznTK05_PS90]

currentmonthyr_list = [currentmonthyr_01,
                       currentmonthyr_02,
                       currentmonthyr_03,
                       currentmonthyr_04,
                       currentmonthyr_05,
                       currentmonthyr_06,
                       currentmonthyr_07]

filepath_list = [filepath_01, filepath_02, filepath_03, filepath_04, filepath_05, filepath_06, filepath_07]

In [ ]:
#-----------------------------------------------Creating the polishing filter list of cell column letters and analysis type-------------------------------
#
FT01A_Zn_list = ['B', 'C', 'D', 'Zn']
FT01B_Zn_list = ['B', 'C', 'E', 'Zn']
FT01C_Zn_list = ['B', 'C', 'F', 'Zn']
FT01D_Zn_list = ['B', 'C', 'G', 'Zn']
FT01E_Zn_list = ['B', 'C', 'H', 'Zn']
FT01F_Zn_list = ['B', 'C', 'I', 'Zn']

FT01A_ntu_list = ['B', 'C', 'J', 'ntu']
FT01B_ntu_list = ['B', 'C', 'K', 'ntu']
FT01C_ntu_list = ['B', 'C', 'L', 'ntu']
FT01D_ntu_list = ['B', 'C', 'M', 'ntu']
FT01E_ntu_list = ['B', 'C', 'N', 'ntu']
FT01F_ntu_list = ['B', 'C', 'O', 'ntu']

FT01A_ZDR_list = ['B', 'C', 'J', 'Zn_dr']
FT01B_ZDR_list = ['B', 'C', 'K', 'Zn_dr']
FT01C_ZDR_list = ['B', 'C', 'L', 'Zn_dr']
FT01D_ZDR_list = ['B', 'C', 'M', 'Zn_dr']
FT01E_ZDR_list = ['B', 'C', 'N', 'Zn_dr']
FT01F_ZDR_list = ['B', 'C', 'O', 'Zn_dr']

analysis_type_FT01A_list = [FT01A_Zn_list,FT01A_ntu_list,FT01A_ZDR_list]
analysis_type_FT01B_list = [FT01B_Zn_list,FT01B_ntu_list,FT01B_ZDR_list]
analysis_type_FT01C_list = [FT01C_Zn_list,FT01C_ntu_list,FT01C_ZDR_list]
analysis_type_FT01D_list = [FT01D_Zn_list,FT01D_ntu_list,FT01D_ZDR_list]
analysis_type_FT01E_list = [FT01E_Zn_list,FT01E_ntu_list,FT01E_ZDR_list]
analysis_type_FT01F_list = [FT01F_Zn_list,FT01F_ntu_list,FT01F_ZDR_list]

analysis_type_FT01AtoF_list = [analysis_type_FT01A_list,
                               analysis_type_FT01B_list,
                               analysis_type_FT01C_list,
                               analysis_type_FT01D_list,
                               analysis_type_FT01E_list,
                               analysis_type_FT01F_list]

In [ ]:
#------------------------------------------------------Spare code----------------------------------------------------------
'''
Jan_2020 = LabResults(currentmonthyr_01, filepath_01)
FT01A_Jan2020 = generate_combined_df_by_month(Jan_2020, '105FT01', analysis_type_FT01A_list)
FT01B_Jan2020 = generate_combined_df_by_month(Jan_2020, '105FT01', analysis_type_FT01B_list)
FT01C_Jan2020 = generate_combined_df_by_month(Jan_2020, '105FT01', analysis_type_FT01C_list)
FT01D_Jan2020 = generate_combined_df_by_month(Jan_2020, '105FT01', analysis_type_FT01D_list)
FT01E_Jan2020 = generate_combined_df_by_month(Jan_2020, '105FT01', analysis_type_FT01E_list)
FT01F_Jan2020 = generate_combined_df_by_month(Jan_2020, '105FT01', analysis_type_FT01F_list)
'''
#TK03_2020_DF = append_monthsDF(tk03_months_list)
#--------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Make dataframes for each month of the polishing filter laboratory results then export as excel file
#   The purpose of exporting the files as excel is to be able to minimize the consequences when the program crashes.
make_df_for_each_FT01_by_month_then_export_excel(1, analysis_type_FT01AtoF_list)
make_df_for_each_FT01_by_month_then_export_excel(2, analysis_type_FT01AtoF_list)
make_df_for_each_FT01_by_month_then_export_excel(3, analysis_type_FT01AtoF_list)
make_df_for_each_FT01_by_month_then_export_excel(4, analysis_type_FT01AtoF_list)
make_df_for_each_FT01_by_month_then_export_excel(5, analysis_type_FT01AtoF_list)
make_df_for_each_FT01_by_month_then_export_excel(6, analysis_type_FT01AtoF_list)
make_df_for_each_FT01_by_month_then_export_excel(7, analysis_type_FT01AtoF_list)
#
# Load the exported excel files of FT01A to F from the previous function
Jan2020_FT01AtoF = load_changeColtitle_join_105FT01(1)
Feb2020_FT01AtoF = load_changeColtitle_join_105FT01(2)
Mar2020_FT01AtoF = load_changeColtitle_join_105FT01(3)
Apr2020_FT01AtoF = load_changeColtitle_join_105FT01(4)
May2020_FT01AtoF = load_changeColtitle_join_105FT01(5)
Jun2020_FT01AtoF = load_changeColtitle_join_105FT01(6)
Jul2020_FT01AtoF = load_changeColtitle_join_105FT01(7)

In [ ]:
#
# List the names of the dataframes for polishing filters grouped by month
FT01AtoF_2020_0107_list = [Jan2020_FT01AtoF,
                           Feb2020_FT01AtoF,
                           Mar2020_FT01AtoF,
                           Apr2020_FT01AtoF,
                           May2020_FT01AtoF,
                           Jun2020_FT01AtoF,
                           Jul2020_FT01AtoF]
# Append the dataframes for polishing filters grouped by month into a single dataframe using the append_monthsDF function 
FT01AtoF_2020_0107 = append_monthsDF(FT01AtoF_2020_0107_list)

In [ ]:
# Export the polishing filter dataframe for the whole year to an excel file.
FT01AtoF_2020_0107.to_excel(r'C:\Users\v.t.flores\Documents\FT01AtoF_2020_0107_DF.xlsx')